# Technical Analysis Template: sRoC + Stochastic Oscillator

Buy and sell using oversold and overbought levels of the Stochastic oscillator according the trend. 
The trend indicator is the smooth Rate of Change with WMA as a smooth function.

In [1]:
import qnt.graph as qngraph
import qnt.data as qndata
import qnt.stats as qnstats
import qnt.xr_talib as qnxrtalib

import xarray as xr
import pandas as pd
from qnt.stepper import test_strategy
#import qnt.forward_looking as qnfl

import xarray.ufuncs as xrf

import datetime as dt

## Data

In [2]:
data = qndata.load_data(tail=dt.timedelta(days=5*365), dims=("time", "field", "asset"), forward_order=True)

fetched chunk 1/6 1s
fetched chunk 2/6 2s
fetched chunk 3/6 4s
fetched chunk 4/6 5s
fetched chunk 5/6 6s
fetched chunk 6/6 7s
Data loaded 7s



## Calc output

In [3]:
SROC_POSITIVE_TREND_LEVEL=0.05
SROC_CLOSE_LEVEL=-0.05

STOCH_OVERBOUGHT_LEVEL=92
STOCH_OVERSOLD_LEVEL=31

wma = qnxrtalib.WMA(data.sel(field='close'), 120)
sroc = qnxrtalib.ROCP(wma, 60)

stoch = qnxrtalib.STOCH(data, 8, 3, 3)
k = stoch.sel(field='slowk')
d = stoch.sel(field='slowd')

data_ext = xr.concat([wma, sroc, k, d], pd.Index(['wma', 'sroc', 'k', 'd'], name='field'))
data_ext = xr.concat([data, data_ext], 'field')

weights = data.isel(time=0, field=0)
weights[:] = 0


def step(data):
    latest = data.isel(time=-1)

    is_liquid = latest.sel(field="is_liquid")
    sroc = latest.sel(field='sroc')
    k = latest.sel(field='k')
    d = latest.sel(field='d')

    need_open = xrf.logical_and(
        sroc > SROC_POSITIVE_TREND_LEVEL, 
        xrf.logical_and(k < STOCH_OVERSOLD_LEVEL, d < STOCH_OVERSOLD_LEVEL)
    )
    need_close = xrf.logical_or(
        sroc < SROC_CLOSE_LEVEL, 
        xrf.logical_and(k > STOCH_OVERBOUGHT_LEVEL, d > STOCH_OVERBOUGHT_LEVEL)
    )

    global weights
    
    weights.loc[need_open] = 1
    weights.loc[need_close] = 0
    
    weights.loc[is_liquid == 0] = 0 # prevention of illiquid assets trading

    return (weights / weights.sum('asset')).fillna(0)


output = test_strategy(data_ext, step=step)

Testing started...
Testing progress: 449/1257 5s
Testing progress: 894/1257 10s
Testing complete 14.093945503234863s


## Stats and plots

In [4]:
stat = qnstats.calc_stat(data, output, max_periods=252 * 3)
display(stat.to_pandas().tail())

field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2020-08-13,1.718650,0.009769,0.242372,-0.011136,-0.389934,0.585076,0.141806,1.0,841.0,0.099468,21.348898
2020-08-14,1.708841,-0.005708,0.242317,-0.016780,-0.389934,0.558242,0.135272,1.0,841.0,0.099451,21.330739
2020-08-17,1.734556,0.015048,0.242459,-0.001984,-0.389934,0.582496,0.141231,1.0,842.0,0.099428,21.323562
2020-08-18,1.736825,0.001308,0.242453,-0.000678,-0.389934,0.578960,0.140371,1.0,842.0,0.099424,21.322874
2020-08-19,1.720233,-0.009553,0.242285,-0.010225,-0.389934,0.592895,0.143649,1.0,842.0,0.099429,21.172750


In [5]:
qngraph.make_plot_filled(
    stat.coords['time'].to_pandas(), 
    stat.loc[:, 'equity'].values,  
    color="blue", 
    name="PnL (Equity)", 
    type="log"
)

In [6]:
qngraph.make_plot_filled(
    stat.coords['time'].to_pandas(), 
    stat.loc[:, 'underwater'].values, 
    color="red", 
    name="Underwater Chart", 
    range_max= 0
)

In [7]:
SR_OFFSET = 252 * 3 + 120 + 60 + 8 * 3 * 3
qngraph.make_plot_filled(
    stat.coords['time'].to_pandas()[SR_OFFSET:], 
    stat.loc[:, 'sharpe_ratio'].values[SR_OFFSET:], 
    color="purple", 
    name="Rolling SR"
)

In [8]:
qngraph.make_plot_filled(
    stat.coords['time'].to_pandas(), 
    stat.loc[:, 'bias'].values, 
    color="gray", 
    name="Bias"
)

## Checks

In [9]:
qnstats.print_correlation(output, data)


WARNING! This strategy correlates with other strategies.
The number of systems with a larger Sharpe ratio and correlation larger than 0.8: 1
The max correlation value (with systems with a larger Sharpe ratio): 0.9055071260527578
Current sharpe ratio(3y): 0.592895288234106



## Save output

In [10]:
qndata.write_output(output)

write output: /root/fractions.nc.gz
